#  Working file :  demand file for 7h-9h

###  As I put at the beginning, all users into MnMs and I got only 60% even less users in the outputs of simulation, also, the previous simulation area was too large. Therefore, I needed to focus only on the city of Amsterdam so that we could  reduce significantly the calculate time, that's why I wanted to try in this part to find a way to project all users on the ring road around the city of Amsterdam. 

### We can see the new simulation area via 'simulation/figure/demand_Amsterdam.png'

### In this part I used the endpoints of simulation area that I've defined in the previous work

### - xmin : 625017.9576056905 <=> 52.32518282260862
### - ymin : 5798791.299367087 <=> 4.834477934348526
### - xmax : 634641.3677946266 <=> 52.42619306976802
### - ymax : 5810286.725571055 <=> 4.980217278228295

In [32]:
import pandas as pd
import numpy as np
import os
import itertools
import random
import utm

## Visualize the missing users after the simulation for 9h-11h

### Example with the outputs of simulation for 9h-11h

In [42]:
input_path1 = os.getcwd() + "/OUTPUTS_9h_11h/path.csv"  # outputs path of simulation between 9h-11h
input_path2 = os.getcwd() + "/INPUTS/test_9h_11h.csv"   # inputs of simulation between 9h-11h
Output = pd.read_csv(input_path1, delimiter = ';')
Input = pd.read_csv(input_path2, delimiter = ';')
listID_in = Input['ID'].tolist()
listID_out = Output['ID'].tolist()

In [43]:
# function takes two lists as input
# returns the different part of two list
def diff(l1, l2):   # return a list of lost users
    difference = list(set(l1).symmetric_difference(set(l2)))
    return difference

In [35]:
# function takes as inputs the list of input, output and the input dataframe
# returns the list of missing users in the output
def missingData(inputDataframe, listID_in, listID_out):
    missing = diff(listID_in, listID_out)
    indexUser = [Input.index[Input['ID'] == i].tolist() for i in missing]  # index of missing users
    indexUser = list(itertools.chain.from_iterable(indexUser))
    return Input.iloc[indexUser]

In [39]:
# missing users 
df_missing = missingData(Input, listID_in, listID_out)  
df_missing

,ID,DEPARTURE,ORIGIN,DESTINATION
9930,917515,09:18:14,613738.9128652506 5818141.26617263,629243.7397205335 5804437.454238096
43780,917515,10:53:12,629243.7397205335 5804437.454238096,613738.9128652506 5818141.26617263
13668,983051,09:25:17,630285.1689666857 5823722.328112848,631435.8993262072 5823049.13385241
4350,1048595,09:07:52,622015.0947725042 5773928.925534364,626851.9692397028 5803852.636913614
31168,589845,10:24:39,691913.3587404573 5762532.257194359,687412.3284794632 5761766.715623426
...,...,...,...,...
9813,786404,09:18:01,639430.1537932879 5778427.223724365,642672.2692429221 5773995.791650763
25632,786404,10:09:22,642672.2692429221 5773995.791650763,639430.1537932879 5778427.223724365
11929,1114103,09:21:59,634526.9620972774 5798128.515459549,607382.294792387 5756226.626322996
15049,917501,09:28:00,612791.2251259275 5818655.7754812995,629088.1051609886 5803890.015116696


In [14]:
# Je l'ai pas utilisé finalement 
def bounding(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    l = O + D  # Concatenate 2 2D lists
    coord = np.array([x.split() for x in l])
    Xmin = min(float(x[0]) for x in coord)
    Ymin = min(float(x[1]) for x in coord)
    Xmax = max(float(x[0]) for x in coord)
    Ymax = max(float(x[1]) for x in coord)
    return Xmin, Xmax, Ymin, Ymax

## Project users on the ring road of Amsterdam

In [18]:
# function takes a dataframe of demand as input
# returns the percentage of users' origin and destination points that are inside or outside the simulation area
def in_out_city(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    ID = df.ID.tolist()  ## no use
    array_O = np.array([x.split() for x in O])
    array_D = np.array([x.split() for x in D])
    coord_O = array_O.astype(float)
    coord_D = array_D.astype(float)
    (xmin, ymin, xmax, ymax) = (625017.9576056905, 5798791.299367087, 634641.3677946266, 5810286.725571055)
    In = lambda x, y : x <= xmax and x >= xmin and y <= ymax and y>= ymin  # if between xmin and xmax
    Out = lambda x, y : x > xmax or x < xmin or y > ymax or y < ymin   # if not
    (in_in, in_out, out_in, out_out) = (0, 0, 0, 0)
    for i in range(len(O)):
        if In(coord_O[i][0], coord_O[i][1]) & In(coord_D[i][0], coord_D[i][1]):
            in_in+=1
        elif In(coord_O[i][0], coord_O[i][1]) & Out(coord_D[i][0], coord_D[i][1]):
            in_out+=1
        elif Out(coord_O[i][0], coord_O[i][1]) & In(coord_D[i][0], coord_D[i][1]):
            out_in+=1
        elif Out(coord_O[i][0], coord_O[i][1]) & Out(coord_D[i][0], coord_D[i][1]):
            out_out+=1
    per_in_in = (in_in / len(df))*100
    per_in_out = (in_out / len(df))*100
    per_out_in = (out_in / len(df))*100
    per_out_out = (out_out / len(df))*100
    sum_out = in_out + out_in + out_out
    return per_in_in, per_in_out, per_out_in, per_out_out, sum_out

In [19]:
# funciton takes a list og coordinates as input
# returns a list of coordinates converted from lat lon to utm
def get_utm(l):
    return [[utm.from_latlon(i[0], i[1])[0], utm.from_latlon(i[0], i[1])[1]] for i in l]

In [20]:
# function takes as input coordinates of 2 points
# returns the distance between this 2 points
def distanceSquare(x1, y1, x2, y2):
    return (x1 - x2)**2 + (y1 - y2)**2

In [21]:
# function takes the list of highway coordinates and the coordinates of a point (x,y) as input
# return the nearest highway point to (x,y)
def nearest_highway(highway, x, y):
    h = get_utm(l)    # turn latlong to utm
    dist = [distanceSquare(x, y, i[0], i[1]) for i in h]
    minpos = dist.index(min(dist))  # get index of minimum distance
    return (h[minpos][0], h[minpos][1])

### Modifie the departure or destination coordinates of users

In [23]:
# function takes a dataframe of demand as input
# returns dataframe of demand modified by projecting users' departure or arrival point on the ring road
def all_in_city(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    ID = df.ID.tolist()
    Departure = df.DEPARTURE.tolist()
    Service = df.SERVICE.tolist()
    l_O = [x.split() for x in O]
    l_D = [x.split() for x in D]
    # 7 points on the ring road of the city of Amsterdam
    highway = [[52.32791994290011, 4.913239361372001], [52.348521124104536, 4.969982909959437], [52.38911039306189, 4.967560455680146],
    [52.40771996733094, 4.938153295847306], [52.4233956111038, 4.877273846428637], 
     [52.3960595030744, 4.844561046121149], [52.3841084139694, 4.8444926286081955], [52.338567002294766, 4.841697415785518]]
    index = []
    # endpoints of the simulation area
    (xmin, ymin, xmax, ymax) = (625017.9576056905, 5798791.299367087, 634641.3677946266, 5810286.725571055)
    In = lambda x, y : x <= xmax and x >= xmin and y <= ymax and y>= ymin  # if between xmin and xmax
    Out = lambda x, y : x > xmax or x < xmin or y > ymax or y < ymin   # if not
    remove = lambda l : [l.pop(x) for x in sorted(index, reverse = True)]
    combine = lambda l : ['%s %s' % (l[x][0], l[x][1]) for x in range(len(l))]
    for i in range(len(l_O)):
        # if one of the departure or arrival point is outside the city area
        if not (In(float(l_O[i][0]), float(l_O[i][1])) & In(float(l_D[i][0]), float(l_D[i][1]))):
            # if departure point is inside and arrival point is outside the city area
            if In(float(l_O[i][0]), float(l_O[i][1])) & Out(float(l_D[i][0]), float(l_D[i][1])):
                (X, Y) = nearest_highway(highway, float(l_D[i][0]),  float(l_D[i][1]))
                l_D[i][0] = str(X)
                l_D[i][1] = str(Y)
            # if departure point is outside and arrival point is inside the city area
            elif Out(float(l_O[i][0]), float(l_O[i][1])) & In(float(l_D[i][0]), float(l_D[i][1])):
                (X, Y) = nearest_highway(highway, float(l_O[i][0]),  float(l_O[i][1]))
                l_O[i][0] = str(X)
                l_O[i][1] = str(Y)
            # if both departure and arrival points are outside the city area
            else:        # out_out -> remove all
                index.append(i)
    remove(ID)
    remove(l_O) 
    remove(l_D)
    remove(Departure)
    remove(Service)
    l_O = combine(l_O)
    l_D = combine(l_D)
    tmp = list(zip(ID, Departure, l_O, l_D, Service))
    df = pd.DataFrame(tmp, columns = ['ID', 'DEPARTURE', 'ORIGIN', 'DESTINATION', 'SERVICE'])
    return df

## Modify the demand for 7h-9h and use this demand for the simulation

### The origin demand and the percentage of inside, outside the city area

In [45]:
# show the test_7h_9h.csv 
test_7h_9h = pd.read_csv('/Users/xinyun/Documents/Stage4A/demand/test_7h_9h.csv', delimiter = ';')
(a,b,c,d,e) = in_out_city(test_7h_9h)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 24% in_out_city : 11% out_in_city : 15% out_out_city : 50% sum_out : 14496


### The modified demand and the percentage of inside, outside the city area

In [46]:
test_all_in = all_in_city(test_7h_9h)
(a,b,c,d,e) = in_out_city(test_all_in)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 100% in_out_city : 0% out_in_city : 0% out_out_city : 0% sum_out : 0


### Save the modified one for the simulation 

In [47]:
test_all_in.to_csv('/Users/xinyun/Documents/Stage4A/demand/demand_7h_9h_Amsterdam_scenario.csv', sep = ';', index = False)